In [3]:
import os
import requests

scenarios = ["pc", "rcp85", "rcp26"]
return_periods = [500, 200, 100, 50, 25, 5, 2]

folderr = "../workspace/GIRI_raw/"
os.makedirs(folderr, exist_ok=True)

for scenario in scenarios:
    for rp in return_periods:
        url = f"https://hazards-data.unepgrid.ch/global_{scenario}_h{rp}glob.tif"
        filename = os.path.join(folderr, f"flood_{scenario}_{rp}_glob.tif")

        if os.path.exists(filename):
            print(f"[SKIP] {filename} already exists.")
            continue

        print(f"[INFO] Downloading {filename} ...")
        try:
            r = requests.get(url, timeout=60)
            if r.status_code == 200:
                with open(filename, "wb") as f:
                    f.write(r.content)
                print(f"[OK] Saved {filename}")
            else:
                print(f"[WARN] URL not found or unavailable: {url} (status {r.status_code})")
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] Failed to download {url}: {e}")

[SKIP] ../workspace/GIRI_raw/flood_pc_500_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_pc_200_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_pc_100_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_pc_50_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_pc_25_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_pc_5_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_pc_2_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp85_500_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp85_200_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp85_100_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp85_50_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp85_25_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp85_5_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp85_2_glob.tif already exists.
[SKIP] ../workspace/GIRI_raw/flood_rcp26_500_glob.tif

# setup


In [5]:
import os
import numpy as np
import xarray as xr
import rioxarray as rxr  # make sure: pip/conda install rioxarray
from tqdm import tqdm

# ---------------- config ----------------
scenarios = ["pc", "rcp85", "rcp26"]
return_periods = [500, 200, 100, 50, 25, 5, 2]

# map to human-readable names (as a coord)
dict_scenarios = {
    "pc": "present",
    "rcp85": "rcp85",
    "rcp26": "rcp26",
}

# crop extent (lon_min, lon_max, lat_max, lat_min)
LON_MIN, LON_MAX = -75.0, -30.0
LAT_MAX, LAT_MIN = 10.0, -35.0

# chunk sizes for dask/xarray and NetCDF
CHUNK_LAT, CHUNK_LON = 512, 512

# ----------------------------------------
list_da = []

# Create total count for progress bar
total_files = len(scenarios) * len(return_periods)
pbar = tqdm(total=total_files, desc="Loading files", unit="file")

for scenario in scenarios:
    for rp in return_periods:
        fpath = os.path.join(folderr, f"flood_{scenario}_{rp}_glob.tif")
        
        if not os.path.exists(fpath):
            pbar.set_postfix({"status": f"Missing: {scenario}_{rp}"})
            pbar.update(1)
            continue

        pbar.set_postfix({"status": f"Loading: {scenario}_{rp}"})
        
        try:
            da = rxr.open_rasterio(
                fpath,
                chunks={"x": CHUNK_LON, "y": CHUNK_LAT},
                masked=True,
            )
        except Exception as e:
            pbar.set_postfix({"status": f"ERROR: {scenario}_{rp}"})
            pbar.update(1)
            continue

        # squeeze band -> 2D (y,x)
        if "band" in da.dims and da.sizes["band"] == 1:
            da = da.squeeze("band", drop=True)

        # ensure CRS; many global hazard tifs are EPSG:4326
        if da.rio.crs is None:
            da = da.rio.write_crs("EPSG:4326")

        # crop (note: y usually descends; slice handles either order)
        da = da.sel(x=slice(LON_MIN, LON_MAX), y=slice(LAT_MAX, LAT_MIN))

        # rename dims/var
        da = da.rename({"x": "lon", "y": "lat"})
        da.name = "flood_depth"

        # set dtype (float32 is compact and typical for depths)
        da = da.astype("float32")

        # expand coords with scenario + return_period
        da = da.expand_dims(
            {
                "GWL": [dict_scenarios.get(scenario, scenario)],
                "return_period": [rp],
                "ensemble": ["mean"]
            }
        )

        list_da.append(da)
        pbar.update(1)

pbar.close()

# guard: anything loaded?
if not list_da:
    raise RuntimeError("No rasters were found/loaded. Check file names and folder.")

# combine by coords into one cube
print("\n[INFO] Combining arrays…")
with tqdm(total=1, desc="Combining") as pbar:
    da_all = xr.combine_by_coords(list_da, combine_attrs="override")
    pbar.update(1)

# order dims - IMPORTANT: This order matches load_nc_hazards_with_metadata expectations
# The function expects: (ensemble, GWL, return_period, lat, lon)
print("[INFO] Transposing dimensions…")

# combine_by_coords returns a Dataset when all DataArrays have the same name
# Just transpose it directly
ds = da_all.transpose("ensemble", "GWL", "return_period", "lat", "lon")

# set some attrs
ds["flood_depth"].attrs.update(
    {
        "long_name": "Flood depth",
        "units": "m",
        "source": "UNEP GRID hazards data",
        "note": "Cropped to South America; scenarios labelled by human-readable names.",
    }
)
ds["GWL"].attrs["description"] = "Scenario label"
ds["return_period"].attrs["units"] = "years"
ds["ensemble"].attrs["description"] = "Ensemble statistics"

# rechunk uniformly for writing
ds = ds.chunk({ "ensemble": 1, "GWL": 1, "return_period": 1, "lat": CHUNK_LAT, "lon": CHUNK_LON})

# encode & save
encoding = {
    "flood_depth": {
        "zlib": True,
        "complevel": 4,
        "dtype": "float32",
        "chunksizes": (1, 1, 1, CHUNK_LAT, CHUNK_LON),  # Note: 5 dims now (ensemble, GWL, return_period, lat, lon)
        "_FillValue": np.float32(np.nan),  # keep NaN for no-data
    }
}


Loading files: 100%|██████████| 21/21 [00:46<00:00,  2.20s/file, status=Loading: rcp26_2]  



[INFO] Combining arrays…


Combining: 100%|██████████| 1/1 [02:15<00:00, 135.73s/it]


[INFO] Transposing dimensions…


In [8]:
out_nc = "../workspace/hazards/Flood/depth/ensemble_return_period.nc"
print(f"\n[INFO] Writing NetCDF -> {out_nc}")
print("[INFO] This may take several minutes...")
ds.to_netcdf(out_nc, encoding=encoding)
print("\n[OK] Done! File written successfully.")


[INFO] Writing NetCDF -> ../workspace/hazards/Flood/depth/ensemble_return_period.nc
[INFO] This may take several minutes...

[OK] Done! File written successfully.
